In [ ]:
import sys
torch_version = "2.2.0" 
url = f"https://data.pyg.org/whl/torch-{torch_version}+cpu.html"

!{sys.executable} -m pip install torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f {url}


In [ ]:
!pip install pdfminer.six


In [ ]:
!pip install --upgrade gensim
!pip install torch_geometric
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__.split('+')[0])")+cpu.html

!pip install PyPDF2 python-docx docx2txt
!pip install --user gensim


In [ ]:
!pip install --user sentence-transformers


In [ ]:
import sys
!{sys.executable} -m pip install sentence-transformers


In [118]:
from sentence_transformers import SentenceTransformer

In [119]:
import pandas as pd

In [120]:
df_jobs=pd.read_csv("scraped_data_jobs.csv")

In [ ]:
df_jobs

In [ ]:
df_jobs["description"][0]

In [ ]:
num_nan = df_jobs['salary'].isna().sum()
print("Number of NaN in salary:", num_nan)


ratio_nan = df_jobs['salary'].isna().mean()
print(f"NaN ratio: {ratio_nan:.2%}")


In [ ]:
df_jobs["salary"] = df_jobs["salary"].fillna("unknown")


In [ ]:
df_jobs

In [ ]:
import json
import pandas as pd
from pandas import json_normalize

def load_json_to_df(path):
    try:
        return pd.read_json(path, lines=True)
    except ValueError:
        pass

    # Regular JSON (list or dict)
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)

    if isinstance(obj, list):
        # list of dicts
        return json_normalize(obj)
    elif isinstance(obj, dict):
       
        for k, v in obj.items():
            if isinstance(v, list):
                return json_normalize(v)
       
        return json_normalize(obj)


path = "Entity Recognition in Resumes.json" 
df = load_json_to_df(path)


print(df.shape)
df.head()


In [ ]:
import sys
!{sys.executable} -m pip install python-dotenv



In [ ]:
import sys
!{sys.executable} -m pip install --quiet openai python-dotenv


In [ ]:
import os, re, json, time
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # reads .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))



In [ ]:
from openai import OpenAI
import os, json
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

EXTRACTION_SCHEMA = {
  "name": "extract_cv",
  "description": "Extract skills, highest degree, and experience from a resume.",
  "parameters": {
    "type": "object",
    "additionalProperties": False,
    "properties": {
      "technical_skills": {"type": "array", "items": {"type": "string"}},
      "non_technical_skills": {"type": "array", "items": {"type": "string"}},
      "highest_degree": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
          "degree": {"type": "string"},
          "field": {"type": "string"},
          "institution": {"type": "string"}
        }

      },
      "experience": {
        "type": "array",
        "items": {
          "type": "object",
          "additionalProperties": False,
          "properties": {
            "title": {"type": "string"},
            "company": {"type": "string"},
            "start": {"type": "string"},
            "end": {"type": "string"},
            "location": {"type": "string"}
          }
        
        }
      }
    }
  
  }
}

SYSTEM_PROMPT = (
  "You extract structured data from resumes. "
  "Return concise, deduplicated skills, but keep them ATOMIC and SPECIFIC: list concrete tools, languages, "
  "frameworks, libraries, services, platforms, databases, and methodologies (e.g., 'C++', 'Java', 'React', "
  "'Docker', 'Kubernetes', 'AWS', 'GCP', 'Terraform', 'PostgreSQL', 'Spark', 'CI/CD', 'REST'). "
  "Also extract non-technical (soft) skills from sections like 'About me'/'Summary' "
  "(e.g., 'communication', 'teamwork', 'leadership', 'motivated', 'ambitious', 'hardworking'). "
  "Normalize technical names (e.g., 'C++', 'AWS'); lowercase is OK for others. "
  "If something is missing, return empty strings or empty arrays."
)


def extract_one_cv(cv_text, model="gpt-4o-mini"):
    resp = client.chat.completions.create(
        model=model,
        messages=[
          {"role": "system", "content": SYSTEM_PROMPT},
          {"role": "user", "content": f"Resume:\n```\n{cv_text}\n```"}
        ],
        tools=[{"type": "function", "function": EXTRACTION_SCHEMA}],
        tool_choice={"type": "function", "function": {"name": "extract_cv"}},
        temperature=0
    )
    msg = resp.choices[0].message
    if not getattr(msg, "tool_calls", None):
        return {"technical_skills":[], "non_technical_skills":[],
                "highest_degree":{"degree":"","field":"","institution":""},
                "experience":[]}
    tool_call = msg.tool_calls[0]
    data = json.loads(tool_call.function.arguments)

    data.setdefault("technical_skills", [])
    data.setdefault("non_technical_skills", [])
    hd = data.setdefault("highest_degree", {})
    hd.setdefault("degree",""); hd.setdefault("field",""); hd.setdefault("institution","")
    data.setdefault("experience", [])
    for e in data["experience"]:
        e.setdefault("title",""); e.setdefault("company","")
        e.setdefault("start",""); e.setdefault("end",""); e.setdefault("location","")
    return data


In [ ]:
import time
import pandas as pd
from dateutil import parser
from datetime import datetime


def clean_str(s):
    return s.strip() if isinstance(s, str) else ""   

# Parse a date string into a datetime object, return None if fails.
def parse_when(s):
    if not isinstance(s, str) or not s.strip():
        return None
    try:
        return parser.parse(s, fuzzy=True)
    except Exception:
        return None

def months_between(start, end):
    if start is None and end is None:
        return 0  
    if start is None:
        return 0
    if end is None:  
        end = datetime.now()
    return max(0, (end.year - start.year) * 12 + (end.month - start.month))


In [ ]:
import re
import pandas as pd
from collections import Counter
from sentence_transformers import SentenceTransformer
import numpy as np

def ensure_list(x):
    if x is None:
        return []
    try:
        if pd.isna(x):
            return []
    except Exception:
        pass
    if isinstance(x, (list, tuple, set)):
        return list(x)
    return [x]

def norm_skill(s):
    return str(s).strip().lower()

# typical soft seeds
SOFT_SEEDS = {
    "communication","teamwork","collaboration","leadership",
    "problem solving","critical thinking","analytical skills",
    "adaptability","creativity","time management","attention to detail",
    "presentation skills","stakeholder management","project management",
    "mentoring","coaching","ownership","accountability","customer focus",
    "negotiation","conflict resolution","decision making",
    "emotional intelligence","work ethic","quick learning ability","hard working",
    "polite","honest","calm"
}


sent_model = SentenceTransformer("all-MiniLM-L6-v2")
soft_seed_list = sorted(SOFT_SEEDS)
soft_seed_embs = sent_model.encode(soft_seed_list, convert_to_numpy=True)
soft_seed_embs = soft_seed_embs / (np.linalg.norm(soft_seed_embs, axis=1, keepdims=True) + 1e-12)

def is_semantic_soft(term: str, threshold: float = 0.55) -> bool:
    if not term: return False
    v = sent_model.encode([term], convert_to_numpy=True)[0]
    v = v / (np.linalg.norm(v) + 1e-12)
    sim = (soft_seed_embs @ v)  # cosine similarity to seed set
    return float(sim.max()) >= threshold

TECHY_RE = re.compile(
    r"(c\+\+|\.net|node(\.js)?|react|angular|ajax|apache|linux|docker|kubernetes|sql|db2|cobol|"
    r"ftp|sftp|tomcat|css3|css|html5?|javascript|java|python|go\b|golang|aws|azure|gcp|spark|"
    r"hadoop|scala|php|ruby|matlab|\br\b|git|jira|jenkins|maven|gradle|file-aid|dfs?sort|ca-7)",
    re.I
)

def build_tech_vocab_from_df(df, df_jobs=None, min_count=1):
    counts = Counter()
   
    if "technical_skills" in df.columns:
        for lst in df["technical_skills"].apply(ensure_list):
            for s in lst:
                counts[norm_skill(s)] += 1
    
    if df_jobs is not None:
        cand_cols = []
        if "extracted_skills" in df_jobs.columns:
            cand_cols.append("extracted_skills")
        for c in df_jobs.columns:
            if re.search(r"(technical|tech|hard)_skills", c, re.I):
                cand_cols.append(c)
        for c in set(cand_cols):
            for lst in df_jobs[c].apply(ensure_list):
                for s in lst:
                    counts[norm_skill(s)] += 1

    vocab = {s for s, cnt in counts.items() if cnt >= min_count}
    vocab -= {norm_skill(s) for s in SOFT_SEEDS}
    return vocab

def fix_misplaced_skills(df, df_jobs=None, min_count=1, use_semantic=True, heuristic_boost=True):
 
    df2 = df.copy(deep=True)
    tech_vocab = build_tech_vocab_from_df(df2, df_jobs=df_jobs, min_count=min_count)

    for col in ("technical_skills", "non_technical_skills"):
        if col not in df2.columns:
            df2[col] = [[] for _ in range(len(df2))]
        df2[col] = df2[col].apply(ensure_list).astype("object")

    moved = 0
    for i, row in df2.iterrows():
        tech_set = {norm_skill(s) for s in row["technical_skills"]}
        soft_src = [norm_skill(s) for s in row["non_technical_skills"]]

        keep_soft = []
        for s in soft_src:
            if not s:
                continue

            # candidate is technical if (in vocab) OR (matches techy regex)
            looks_tech = (s in tech_vocab) or (heuristic_boost and TECHY_RE.search(s))

            is_soft_seed = (s in {norm_skill(x) for x in SOFT_SEEDS})
            is_sem_soft = use_semantic and is_semantic_soft(s)

            if looks_tech and not is_soft_seed and not is_sem_soft:
                tech_set.add(s)
                moved += 1
            else:
                keep_soft.append(s)

        df2.at[i, "technical_skills"] = sorted(tech_set)
        df2.at[i, "non_technical_skills"] = sorted(set(keep_soft))

    print(f"[info] moved {moved} items from non_technical_skills → technical_skills")
    return df2


In [ ]:
def bucket_experience(total_months: int) -> str:
    m = total_months if isinstance(total_months, (int, float)) else 0
    if m < 12:
        return "0-1 years"
    elif m < 36:
        return "1-3 years"
    elif m < 60:
        return "3-5 years"
    else:
        return "5+ years"

In [ ]:
import time
import pandas as pd


def extract_all(df, text_col="content", sleep_s=0.2):
    """
    Returns:
      df_struct: one row per candidate with structured fields (incl. numeric tenure)
      df_triples: (candidate_id, predicate, object) categorical triples, including:
        - knows (tech skill)
        - has_soft (non-technical skill)
        - has_degree (degree)
        - studied_at (institution)
        - in_field (degree specialization)
        - experience_level (bucketized tenure)
    """
    rows = []


    for i, text in enumerate(df[text_col].tolist()):
        try:
            data = extract_one_cv(text) 
        except Exception as e:
            print(f"[warn] row {i} failed: {e}")
            data = {
                "technical_skills": [],
                "non_technical_skills": [],
                "highest_degree": {"degree": "", "field": "", "institution": ""},
                "experience": []
            }

        # skills 
        tech = sorted({ norm_skill(x)
                        for x in data.get("technical_skills", [])
                        if isinstance(x, str) and x.strip() })
        soft = sorted({ x.strip()
                        for x in data.get("non_technical_skills", [])
                        if isinstance(x, str) and x.strip() })

        # degree 
        deg = data.get("highest_degree", {}) or {}
        degree = clean_str(deg.get("degree", ""))
        field  = clean_str(deg.get("field", ""))
        inst   = clean_str(deg.get("institution", ""))

        # experiences -> total tenure
        exps = data.get("experience", []) or []
        total_tenure = 0
        for e in exps:
            start = parse_when(e.get("start", ""))
            end   = parse_when(e.get("end", ""))
            total_tenure += months_between(start, end)

        exp_bucket = bucket_experience(total_tenure)

        #  structured row (numeric + lists) 
        rows.append({
            "candidate_id": i,
            "technical_skills": tech,
            "non_technical_skills": soft,
            "degree": degree,
            "field": field,
            "institution": inst,
            "num_experiences": len(exps),
            "total_tenure_months": total_tenure,
            "experience_level": exp_bucket,   
        })

       
    df_struct  = pd.DataFrame(rows)
 
    return df_struct

# usage 
df_struct = extract_all(df, text_col="content")



In [ ]:
df_fixed = fix_misplaced_skills(df_struct, df_jobs=df_jobs, min_count=2, use_semantic=True, heuristic_boost=True)


# df_fixed.loc[[0,1,4], ["technical_skills","non_technical_skills"]]


In [ ]:
display(df_fixed.head())


In [ ]:
df_struct = df_fixed.copy(deep=True)


In [ ]:
display(df_struct.head())

In [ ]:
df_struct

In [ ]:
df_jobs

In [ ]:
import ast

# Parse original stringified lists into real Python lists
df_jobs["extracted_skills"] = df_jobs["extracted_skills"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Normalize the skills
df_jobs["extracted_skills_norm"] = df_jobs["extracted_skills"].apply(
    lambda lst: sorted({s.strip().lower() for s in lst if isinstance(s, str)})
)


In [ ]:
cand_skills_norm = [
    norm_skill(s)
    for lst in df_struct["technical_skills"]
    for s in (lst if isinstance(lst, list) else [])
]

job_skills_norm = [
    norm_skill(s)
    for lst in df_jobs["extracted_skills_norm"]
    for s in lst
]

all_skills = sorted({s for s in cand_skills_norm + job_skills_norm if s})
skill2id = {s:i for i,s in enumerate(all_skills)}
print("Skill vocab size:", len(all_skills))

In [ ]:
import numpy as np
import pandas as pd

cand_ids = sorted(df_struct["candidate_id"].unique().tolist())
cand2row = {cid:i for i, cid in enumerate(cand_ids)}  # 0..C-1 row index for features

job_ids = df_jobs['id'].tolist() if 'id' in df_jobs else list(range(len(df_jobs)))
job2id = {j:i for i,j in enumerate(job_ids)}

all_soft   = sorted({s for lst in df_struct["non_technical_skills"] for s in lst})
soft2id    = {s:i for i,s in enumerate(all_soft)}

# Technical BoW
C = len(cand_ids)
cand_bow = np.zeros((C, len(all_skills)), dtype=np.float32)
for _, row in df_struct.iterrows():
    cid = int(row["candidate_id"])
    r   = cand2row[cid]       
    for s in row["technical_skills"]:
        ns  = norm_skill(s)
        sid = skill2id.get(ns)
        if sid is not None:
            cand_bow[r, sid] = 1.0
#Non-technical BoW
cand_soft_bow = np.zeros((C, len(all_soft)), dtype=np.float32)
if len(all_soft) > 0:
    for cid in cand_ids:
        r = cand2row[cid]       
        for s in df_struct.set_index('candidate_id').loc[cid, 'non_technical_skills']:
            ns = norm_skill(s)
            if ns in soft2id:
                cand_soft_bow[r, soft2id[ns]] = 1.0


tenure_train_max = float(df_struct['total_tenure_months'].max())
# Tenure scalar
tenure = (df_struct.set_index('candidate_id')
                    .loc[cand_ids]['total_tenure_months']
                    .to_numpy()
                    .reshape(-1,1))
tenure = tenure / (tenure_train_max + 1e-6)  


# Final candidate.x = [CV_text_emb || tech-skill BoW || soft-skill BoW || tenure]


In [ ]:
import numpy as np
import pandas as pd
import torch

#Candidate features (emb + BoW + soft BoW + tenure)
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer("all-MiniLM-L6-v2")

# Build a list of CV texts aligned to cand_ids
cv_texts = [df.loc[df_struct.index[df_struct['candidate_id']==cid][0], 'content'] for cid in cand_ids]
cv_emb = sent_model.encode(cv_texts, convert_to_numpy=True)   # shape [C, D_txt]

# Final candidate features
parts = [cv_emb, cand_bow, tenure]
if len(all_soft) > 0:
    parts.insert(2, cand_soft_bow) 
candidate_x = np.concatenate(parts, axis=1)







In [ ]:
D_txt_train = cv_emb.shape[1]              
S_train     = len(skill2id)
Q_train     = len(soft2id)
print("Spec:", D_txt_train, S_train, Q_train, "->",
      D_txt_train + S_train + Q_train + 1)






In [ ]:
# save frozen artifacts
import pickle, numpy as np
with open("skill2id_train.pkl","wb") as f: pickle.dump(skill2id, f)
with open("soft2id_train.pkl","wb")  as f: pickle.dump(soft2id, f)
with open("tenure_train_max.json","w") as f: f.write(json.dumps({"tenure_train_max": tenure_train_max}))


In [ ]:
all_skills


In [ ]:
# Text embeddings (float32 + row L2)
job_desc = df_jobs['description'].tolist()
job_emb = sent_model.encode(job_desc, convert_to_numpy=True).astype(np.float32)
job_emb /= (np.linalg.norm(job_emb, axis=1, keepdims=True) + 1e-12)


J, S = len(df_jobs), len(skill2id)
job_bow = np.zeros((J, S), dtype=np.float32)
for j_idx, lst in enumerate(df_jobs["extracted_skills_norm"]):
    for s in set(lst or []):                        
        sid = skill2id.get(norm_skill(s))            
        if sid is not None:
            job_bow[j_idx, sid] = 1.0


bow_block = job_bow  


w_txt, w_bow = 1.0, 1.0
job_x = np.concatenate([w_txt*job_emb, w_bow*bow_block], axis=1).astype(np.float32)



In [ ]:
# Edge building (candidate→skill, job→skill) 
src_c, dst_s = [], []
for _, row in df_struct.iterrows():
    c_row = cand2row[row['candidate_id']]
    for s in row['technical_skills']:
        sid = skill2id.get(s)
        if sid is not None:
            src_c.append(c_row)
            dst_s.append(sid)
edge_c_has_s = torch.tensor([src_c, dst_s], dtype=torch.long)


src_j, dst_s = [], []
for j_idx, lst in enumerate(df_jobs["extracted_skills_norm"]):
    for s in lst:
        sid = skill2id.get(s)
        if sid is not None:
            src_j.append(j_idx)
            dst_s.append(sid)

edge_j_req_s = torch.tensor([src_j, dst_s], dtype=torch.long)
print("job->requires->skill edges:", edge_j_req_s.shape)  


In [ ]:
# build HeteroData
from torch_geometric.data import HeteroData

import numpy as np

rng = np.random.default_rng(seed=42)
data = HeteroData()
data['candidate'].x = torch.from_numpy(candidate_x) 
data['job'].x       = torch.from_numpy(job_x)     

S = len(skill2id)
skill_x = torch.from_numpy(rng.standard_normal((S, 64)).astype(np.float32))

data['skill'].x   = skill_x        

data[('candidate','has','skill')].edge_index   = edge_c_has_s
data[('skill','had_by','candidate')].edge_index = edge_c_has_s.flip(0)
data[('job','requires','skill')].edge_index    = edge_j_req_s
data[('skill','required_by','job')].edge_index = edge_j_req_s.flip(0)

In [ ]:
print("skill nodes:", int(data['skill'].x.size(0)) if 'skill' in data.node_types else 'n/a')


In [ ]:
print("cv_emb:", cv_emb.shape)           
print("cand_bow:", cand_bow.shape)        
print("cand_soft_bow:", cand_soft_bow.shape if 'cand_soft_bow' in globals() else (C,0))
print("tenure:", tenure.shape)            # [C, 1]
print("candidate_x:", candidate_x.shape)
print("model expects:", int(data['candidate'].x.size(1)))


In [ ]:
print("cv_emb:",  cv_emb.shape,  "job_emb:", job_emb.shape)
print("cand_bow:", cand_bow.shape, "job_bow:", job_bow.shape)


In [ ]:
print("jobs N:", len(df_jobs))
print("extracted_skills dtype:", df_jobs["extracted_skills_norm"].apply(type).value_counts().head(5))
print("First 3 raw rows:", df_jobs["extracted_skills_norm"].head(3).tolist())
print("Nulls:", df_jobs["extracted_skills_norm"].isna().sum())


In [ ]:
df_jobs = df_jobs.drop(columns=["extracted_skills"])

print(df_jobs.columns)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, SAGEConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CJGNN(nn.Module):
    def __init__(self, hid=128, drop=0.2):
        super().__init__()
        self.conv1 = HeteroConv({
            ('candidate','has','skill'):        SAGEConv((-1,-1), hid),
            ('skill','had_by','candidate'):     SAGEConv((-1,-1), hid),
            ('job','requires','skill'):         SAGEConv((-1,-1), hid),
            ('skill','required_by','job'):      SAGEConv((-1,-1), hid),
        }, aggr='sum')
        self.conv2 = HeteroConv({
            ('candidate','has','skill'):        SAGEConv((hid,hid), hid),
            ('skill','had_by','candidate'):     SAGEConv((hid,hid), hid),
            ('job','requires','skill'):         SAGEConv((hid,hid), hid),
            ('skill','required_by','job'):      SAGEConv((hid,hid), hid),
        }, aggr='sum')
        self.drop = nn.Dropout(drop)

    def forward(self, x_dict, edge_index_dict):
        x = self.conv1(x_dict, edge_index_dict)
        x = {k: self.drop(F.relu(v)) for k,v in x.items()}
        x = self.conv2(x, edge_index_dict)
        x = {k: self.drop(F.relu(v)) for k,v in x.items()}
        return x

def link_logits(cand_emb, job_emb, scale=20.0):
    c = F.normalize(cand_emb, dim=-1)
    j = F.normalize(job_emb, dim=-1)
    return 20 * (c * j).sum(dim=-1)

model = CJGNN(hid=128, drop=0.2).to(device)
opt   = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

x_dict  = {k: v.to(device).float() for k, v in data.x_dict.items()}
e_index = {k: v.to(device)         for k, v in data.edge_index_dict.items()}


In [ ]:
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
C = len(cand_ids) 
J = len(job_ids)    

sim_text  = cosine_similarity(cv_emb, job_emb)     # shape [C, J]
sim_skill = cosine_similarity(cand_bow, job_bow)   # shape [C, J]

score0 = 0.6*sim_text + 0.4*sim_skill

pos_edges = []
K = 5  # take top-5 weak positives per candidate
for c in range(C):
    topj = np.argsort(score0[c])[-K:]
    for j in topj:
        pos_edges.append((c, j))
pos_edges = np.array(pos_edges, dtype=np.int64)

np.random.shuffle(pos_edges)
n = len(pos_edges)
n_tr = int(0.8*n)
n_va = int(0.1*n)
train_pos = pos_edges[:n_tr]
val_pos = pos_edges[n_tr:n_tr+n_va]
test_pos = pos_edges[n_tr+n_va:]

def sample_negs(num, C, J, forbid=set()):
    res, used = [], set(forbid)
    while len(res) < num:
        c = np.random.randint(0, C)
        j = np.random.randint(0, J)
        if (c,j) not in used:
            used.add((c,j))
            res.append((c,j))
    return np.array(res, dtype=np.int64)

train_neg = sample_negs(len(train_pos), C, J, forbid=set(map(tuple, train_pos)))
val_neg   = sample_negs(len(val_pos),   C, J)
test_neg  = sample_negs(len(test_pos),  C, J)


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics.pairwise import cosine_similarity

device = next(model.parameters()).device

text_prior  = cosine_similarity(cv_emb, job_emb)        # [C, J] #semantic similarity
den = np.clip(cand_bow.sum(1, keepdims=True), 1, None) 
skill_prior = (cand_bow @ job_bow.T) / den              # fraction of cand skills present in job
prior_np = 0.6*text_prior + 0.4*skill_prior
prior_t  = torch.tensor(prior_np, dtype=torch.float32, device=device)


def link_logits_with_prior(cand_emb, job_emb, c_idx, j_idx):
    base = link_logits(cand_emb, job_emb)               # scaled cosine logit
    return base + prior_t[c_idx, j_idx]                 # still a logit


@torch.no_grad()
def forward_embeddings():
    model.eval()
    out = model(x_dict, e_index)
    return out['candidate'], out['job']  # embeddings [C,D], [J,D]


@torch.no_grad()
def sample_hard_negs(pos_pairs, per_pos=1, all_pos_pairs=None):
    ce, je = forward_embeddings()
    scores = (F.normalize(ce, dim=-1) @ F.normalize(je, dim=-1).t()).cpu().numpy()

    pos_set = set(map(tuple, all_pos_pairs)) if all_pos_pairs is not None else set(map(tuple, pos_pairs))

    neg_pairs = []
    for (c, j_pos) in pos_pairs:
        order = np.argsort(scores[c])[::-1]
        for j in order:
            if j != j_pos and (c, j) not in pos_set:
                neg_pairs.append((c, int(j)))
                break
    return np.array(neg_pairs, dtype=np.int64)


def run_batch_bpr(pos_pairs, neg_pairs, use_prior=True):
    model.train(); opt.zero_grad()
    out = model(x_dict, e_index)
    ce, je = out['candidate'], out['job']

    pos_pairs = torch.tensor(pos_pairs, device=device)
    neg_pairs = torch.tensor(neg_pairs, device=device)
    c_pos, j_pos = pos_pairs[:,0], pos_pairs[:,1]
    c_neg, j_neg = neg_pairs[:,0], neg_pairs[:,1]

    if use_prior:
        s_pos = link_logits_with_prior(ce[c_pos], je[j_pos], c_pos, j_pos)
        s_neg = link_logits_with_prior(ce[c_neg], je[j_neg], c_neg, j_neg)
    else:
        s_pos = link_logits(ce[c_pos], je[j_pos])
        s_neg = link_logits(ce[c_neg], je[j_neg])

    # BPR (pairwise) loss: push positives above negatives
    loss = -F.logsigmoid(s_pos - s_neg).mean()
    loss.backward(); opt.step()

    with torch.no_grad():
        pair_acc = (s_pos > s_neg).float().mean().item()  # how often pos>neg
    return float(loss.item()), pair_acc


@torch.no_grad()
def edge_scores(pairs_np, use_prior=True):
    ce, je = forward_embeddings()
    pairs = torch.as_tensor(pairs_np, device=ce.device)
    c, j = pairs[:, 0], pairs[:, 1]

    base = link_logits(ce[c], je[j])       # scaled cosine logits
    logits = base + prior_t[c, j] if use_prior else base
    return torch.sigmoid(logits).cpu().numpy()

@torch.no_grad()
def edge_logits(pairs_np, use_prior=True):
    model.eval()
    out = model(x_dict, e_index)
    ce, je = out['candidate'], out['job']
    pairs = torch.as_tensor(pairs_np, device=ce.device)
    c, j  = pairs[:,0], pairs[:,1]
    base  = link_logits(ce[c], je[j])                 # scaled cosine logit
    logits = base + prior_t[c, j] if use_prior else base
    return logits.cpu().numpy()


def eval_roc_ap(pos_pairs, neg_pairs, use_prior=True):
    if len(pos_pairs) == 0 or len(neg_pairs) == 0:
        return float('nan'), float('nan')
    pos = edge_scores(pos_pairs, use_prior)
    neg = edge_scores(neg_pairs, use_prior)
    y_true   = np.concatenate([np.ones_like(pos), np.zeros_like(neg)])
    y_scores = np.concatenate([pos,               neg])
    return roc_auc_score(y_true, y_scores), average_precision_score(y_true, y_scores)


EPOCHS = 120; USE_PRIOR = True;
for epoch in range(1, EPOCHS+1):
    neg_hard = sample_hard_negs(train_pos, per_pos=1)             # hard negatives
    loss, pair_acc = run_batch_bpr(train_pos, neg_hard, use_prior=USE_PRIOR)

    if epoch % 10 == 0:
        roc, ap = eval_roc_ap(val_pos, val_neg, use_prior=USE_PRIOR)
        print(f"... BPR loss {loss:.4f} ... ROC-AUC {roc:.3f} ... AP {ap:.3f} ... ")



In [ ]:
# Final test metrics
roc_t, ap_t = eval_roc_ap(test_pos, test_neg, use_prior=USE_PRIOR)

print(
    f"[TEST] ROC-AUC {roc_t:.3f} | AP {ap_t:.3f} "
)


In [ ]:
torch.save(model.state_dict(), "cjgnn.pth")

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

USE_PRIOR = True

# Build labels for validation edges
y_val_pos = np.ones(len(val_pos), dtype=np.int64)
y_val_neg = np.zeros(len(val_neg), dtype=np.int64)

# Get raw logits from the current (trained) model
logits_val_pos = edge_logits(val_pos, use_prior=USE_PRIOR)
logits_val_neg = edge_logits(val_neg, use_prior=USE_PRIOR)

logits_val = np.concatenate([logits_val_pos, logits_val_neg]).reshape(-1, 1)
labels_val = np.concatenate([y_val_pos,      y_val_neg])


In [ ]:
calibrator = LogisticRegression(solver="lbfgs", max_iter=2000)  
calibrator.fit(logits_val, labels_val)


In [ ]:
df_struct

In [ ]:
cv_path = r"C:\Users\Filip\Downloads\AlexJohnsonResume.pdf"

In [ ]:
import os, re, json
from PyPDF2 import PdfReader
import docx2txt
import pandas as pd

# Read text from a .pdf or .docx
def extract_text_from_file(path: str) -> str:
    p = path.lower()
    if p.endswith(".pdf"):
        reader = PdfReader(path)
        text = "\n".join((page.extract_text() or "") for page in reader.pages)
    elif p.endswith(".docx"):
        text = docx2txt.process(path) or ""
    else:
        raise ValueError("Unsupported file type (.pdf or .docx only)")
    text = re.sub(r'\s+\n', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()

# Call LLM extractor on the file text 
def extract_cv_from_uploaded_file(path: str, model="gpt-4o-mini"):
    cv_text = extract_text_from_file(path)
    if not cv_text:
        return {
            "technical_skills": [],
            "non_technical_skills": [],
            "highest_degree": {"degree":"", "field":"", "institution":""},
            "experience": []
        }
    
    return extract_one_cv(cv_text, model=model)


def extract_single_cv_to_struct(path: str, candidate_id: int, df_jobs: pd.DataFrame | None = None):

    data = extract_cv_from_uploaded_file(path)

    # skills
    tech = sorted({ norm_skill(s) for s in ensure_list(data.get("technical_skills")) if s })
    soft = sorted({ s.strip() for s in ensure_list(data.get("non_technical_skills")) if s })

    # degree
    deg = data.get("highest_degree", {}) or {}
    degree = clean_str(deg.get("degree", ""))
    field  = clean_str(deg.get("field", ""))
    inst   = clean_str(deg.get("institution", ""))

    # tenure
    exps = ensure_list(data.get("experience"))
    total_tenure = 0
    for e in exps:
        if not isinstance(e, dict): continue
        start = parse_when(e.get("start", ""))
        end   = parse_when(e.get("end", ""))
        total_tenure += months_between(start, end)
    exp_bucket = bucket_experience(total_tenure)

    df_struct_single = pd.DataFrame([{
        "candidate_id": candidate_id,
        "technical_skills": tech,
        "non_technical_skills": soft,
        "degree": degree,
        "field": field,
        "institution": inst,
        "num_experiences": len(exps),
        "total_tenure_months": int(total_tenure),
        "experience_level": exp_bucket,
    }])


    return df_struct_single


In [ ]:
df_struct_single = extract_single_cv_to_struct(cv_path, candidate_id=1234)


In [ ]:
df_struct_single


In [ ]:
# Load your CV file text
from pdfminer.high_level import extract_text
cv_text = extract_text(cv_path) or ""
print(cv_text[:500]) 

In [ ]:
# Load frozen artifacts
import pickle, json
with open("skill2id_train.pkl","rb") as f: skill2id = pickle.load(f)
with open("soft2id_train.pkl","rb")  as f: soft2id  = pickle.load(f)   # or set None if you didn't use soft in training
tenure_train_max = json.load(open("tenure_train_max.json"))["tenure_train_max"]

S_train = len(skill2id)
Q_train = len(soft2id) if soft2id else 0
D_txt_train = cv_emb.shape[1]  # must match training encoder/projection
feat_dim_expected = int(data['candidate'].x.size(1))  # 2094


In [ ]:
S_train

In [ ]:
Q_train

In [ ]:
D_txt_train

In [ ]:
feat_dim_expected

In [ ]:
import re

def fix_broken_letter_spacing(text: str) -> str:
    # collapse sequences of single-letter tokens: "E m a i l" -> "Email"
    def _join_run(run):
        # join letters, keep an initial capital if present
        word = "".join(run)
        return word

    tokens = text.split()
    out, run = [], []
    for t in tokens:
        if len(t) == 1 and re.match(r"[A-Za-z]", t):
            run.append(t)
        else:
            if run:
                out.append(_join_run(run)); run = []
            out.append(t)
    if run:
        out.append(_join_run(run))
    return re.sub(r'\s+', ' ', " ".join(out)).strip()


In [ ]:
cv_text = extract_text_from_file(cv_path)
cv_text = fix_broken_letter_spacing(cv_text)
print("chars:", len(cv_text))
print(cv_text[:400])


In [ ]:
def embed_cv_text(cv_text, model, max_tokens=500):
    toks = cv_text.split()
    chunks = [" ".join(toks[i:i+max_tokens]) for i in range(0, len(toks), max_tokens)]
    embs = sent_model.encode(chunks, convert_to_numpy=True)
    return embs.mean(axis=0, keepdims=True)  # (1, D)




In [ ]:
import numpy as np

def build_aligned_candidate_row(
    cv_emb_new: np.ndarray,                 # shape (1, D_txt_new)
    cand_tech_norm: list,                  
    skill2id: dict,                      
    all_soft_len_expected: int = None,   
    soft2id: dict | None = None,        
    cand_soft_norm: list | None = None,  
    tenure_months_new: float = 0.0,        
    tenure_train_max: float = None,         
    data=None                            
):


    #  Text embedding block
    if cv_emb_new.ndim == 1:
        cv_emb_new = cv_emb_new.reshape(1, -1)  # ensure 2D
    blocks = [cv_emb_new.astype(np.float32)]

    # Technical skills BoW
    tech_bow = np.zeros((1, len(skill2id)), dtype=np.float32)
    for s in (cand_tech_norm or []):
        sid = skill2id.get(s)
        if sid is not None:
            tech_bow[0, sid] = 1.0
    blocks.append(tech_bow)

    #  Soft skills BoW 
    if all_soft_len_expected is None:
        all_soft_len_expected = len(soft2id) if soft2id else 0
    soft_bow = np.zeros((1, all_soft_len_expected), dtype=np.float32)
    if cand_soft_norm and soft2id:
        for s in cand_soft_norm:
            sid = soft2id.get(s)
            if sid is not None:
                soft_bow[0, sid] = 1.0
    blocks.append(soft_bow)

    # Tenure
    tenure_scaled = (
        np.array([[tenure_months_new]], dtype=np.float32) /
        (tenure_train_max + 1e-6 if tenure_train_max else 1.0)
    )
    blocks.append(tenure_scaled)

    #  Concatenate all blocks 
    cand_row = np.concatenate(blocks, axis=1)
    return cand_row


In [ ]:
cv_emb_single = embed_cv_text(cv_text, sent_model)   # (1, 384)
# Build the candidate row
cand_row = build_aligned_candidate_row(
    cv_emb_new=cv_emb_single,                      
    cand_tech_norm=df_struct_single.loc[0, "technical_skills"],
    skill2id=skill2id,                                
    all_soft_len_expected=len(soft2id) if soft2id else None,
    soft2id=soft2id,                                      
    cand_soft_norm=df_struct_single.loc[0, "non_technical_skills"],
    tenure_months_new=df_struct_single.loc[0, "total_tenure_months"],
    tenure_train_max=tenure_train_max,                      
    data=data                                             
)


In [ ]:
print(df_struct_single["technical_skills"].iloc[0])
print(df_struct_single["non_technical_skills"].iloc[0])


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData

def score_topk_consistent(
    model,
    data: HeteroData,
    cand_row_np,
    cand_skill_list,
    skill2id_node,
    topk=10,
    device="cpu",
    scale=20.0,         
):

    #  prep model + dtypes/devices 
    dev = torch.device(device)
    model = model.to(dev).eval()

    # clone a light runtime graph
    rt = HeteroData()
    for nt in data.node_types:
        rt[nt].x = data[nt].x.detach().to(dev).float().contiguous()
    for et in data.edge_types:
        rt[et].edge_index = data[et].edge_index.detach().to(dev).long().contiguous()
    rt.metadata()  
    
    # append new candidate row 
    cand_row_np = np.asarray(cand_row_np, dtype=np.float32)
    if cand_row_np.ndim == 1:
        cand_row_np = cand_row_np[None, :]  # (1, D)

    # width check
    D_train = rt['candidate'].x.size(1) #(num_candidates, D_features)
    if cand_row_np.shape[1] != D_train:
        raise ValueError(
            f"Candidate feature width mismatch: got {cand_row_np.shape[1]}, expected {D_train}.\n"
        )

    C_old = rt['candidate'].x.size(0)
    cand_row = torch.from_numpy(cand_row_np).to(dev).float().contiguous()
    rt['candidate'].x = torch.cat([rt['candidate'].x, cand_row], dim=0)

    # map strings -> skill ids;
    if 'skill' in rt.node_types:
        S_nodes = rt['skill'].x.size(0)
        skill_ids = []
        for s in cand_skill_list:
            sid = skill2id_node.get(s)
            if sid is not None and 0 <= sid < S_nodes:
                skill_ids.append(int(sid))
        if skill_ids:
            new_src = torch.full((len(skill_ids),), C_old, dtype=torch.long, device=dev)
            new_dst = torch.tensor(skill_ids, dtype=torch.long, device=dev)

            e_old = rt[('candidate','has','skill')].edge_index
            rt[('candidate','has','skill')].edge_index = torch.cat(
                [e_old, torch.stack([new_src, new_dst], dim=0)], dim=1
            )

            e_old_r = rt[('skill','had_by','candidate')].edge_index
            rt[('skill','had_by','candidate')].edge_index = torch.cat(
                [e_old_r, torch.stack([new_dst, new_src], dim=0)], dim=1
            )

    # forward + scoring 
    with torch.no_grad():
        z_dict = model(rt.x_dict, rt.edge_index_dict)
        z_cand = z_dict['candidate'][C_old:C_old+1]         
        z_job  = z_dict['job']                              

        zc = F.normalize(z_cand, p=2, dim=1)                
        zj = F.normalize(z_job,  p=2, dim=1)                
        cosine = (zc @ zj.T).squeeze(0)                     
        logits = scale * cosine                             
        probs  = torch.sigmoid(logits)                      

        # top-k by logits (desc)
        topk = min(int(topk), logits.numel())
        topk_idx = torch.topk(logits, k=topk).indices.cpu().numpy()

    # numpy outputs (float32)
    return (
        topk_idx,
        cosine.detach().cpu().float().numpy(),
        logits.detach().cpu().float().numpy(),
        probs.detach().cpu().float().numpy(),
    )


In [ ]:
df_struct_single

In [121]:
import numpy as np
import pandas as pd

skill2id_node = {s: i for i, s in enumerate(all_skills)} 
top_idx, cos_scores, logits, probs = score_topk_consistent(
    model, data,
    cand_row_np=cand_row,
    cand_skill_list=[norm_skill(s) for s in df_struct_single.loc[0, "technical_skills"]],
    skill2id_node=skill2id_node,
    topk=10,
    device="cpu",
    scale=20.0
)
logits_all = logits.reshape(-1, 1)           
probs_calibrated_all = calibrator.predict_proba(logits_all)[:, 1]  


recs = (
    df_jobs.iloc[top_idx]
      .assign(
          cosine = np.round(cos_scores[top_idx], 4),
          logit  = np.round(logits[top_idx], 4),
          match_prob = (probs_calibrated_all[top_idx] * 100).round(1)  # calibrated %
      )
      .sort_values("logit", ascending=False)
      .reset_index(drop=True)
)

def categorize_match(p):
    if p >= 75:
        return "Strong"
    elif p >= 60:
        return "Moderate"
    else:
        return "Weak"

recs["match_level"] = recs["match_prob"].apply(categorize_match)

print(recs[["id","title","company_name","cosine","logit","match_prob","match_level"]])



        id                                      title  \
0  2045144    Distributed Cloud | AWS DevOps Engineer   
1  2040720  Senior Consultant - Business Intelligence   
2  2034578                 Senior Lead Java Developer   
3  2022739                 Senior FullStack Developer   
4  2042775                   Senior Software Engineer   
5  2044663                             Cloud Engineer   
6  2028991     Integrations Engineer / Java Developer   
7  2048539                   Java Fullstack Developer   
8  1383531              Senior React Native Developer   
9  2044815                  AWS Cloud DevOps Engineer   

                        company_name  cosine   logit  match_prob match_level  
0                           Devoteam  0.4684  9.3673        98.6      Strong  
1                      Iriumportugal  0.4560  9.1194        98.2      Strong  
2                         UMPISA INC  0.4503  9.0067        98.0      Strong  
3                          Acumatica  0.4459  8.9181    

In [122]:
recs


,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,fetched_category,extracted_skills,cosine,logit,match_prob,match_level
0,2045144,https://remotive.com/remote-jobs/devops/distri...,Distributed Cloud | AWS DevOps Engineer,Devoteam,https://remotive.com/job/2045144/logo,DevOps / Sysadmin,"['AWS', 'python', 'CI/CD', 'terraform', 'ansib...",full_time,2025-08-09T10:52:17,EMEA,NaN,"Company Description At Devoteam, we believe th...",data-analysis,"['AWS', 'Ansible', 'CI/CD', 'Cloud', 'Docker',...",0.4684,9.3673,98.6,Strong
1,2040720,https://remotive.com/remote-jobs/data/senior-c...,Senior Consultant - Business Intelligence,Iriumportugal,https://remotive.com/job/2040720/logo,Data Analysis,"['business intelligence', 'SQL Server']",full_time,2025-07-29T12:50:15,Portugal,NaN,At IRIUM we want you to always chase your dre...,data-analysis,"['Azure', 'Big Data', 'Business Intelligence',...",0.4560,9.1194,98.2,Strong
2,2034578,https://remotive.com/remote-jobs/software-dev/...,Senior Lead Java Developer,UMPISA INC,https://remotive.com/job/2034578/logo,Software Development,['spring boot'],full_time,2025-07-19T08:51:13,Philippines,NaN,"At Umpisa Inc. , our mission is to make the Ph...",software-dev,"['API', 'AWS', 'Angular', 'Azure', 'CI/CD', 'C...",0.4503,9.0067,98.0,Strong
3,2022739,https://remotive.com/remote-jobs/software-dev/...,Senior FullStack Developer,Acumatica,https://remotive.com/job/2022739/logo,Software Development,"['.Net', 'C#', 'javascript', 'sql']",full_time,2025-06-19T12:50:52,Serbia,NaN,Company Description Acumatica is an American l...,software-dev,"['.NET', 'Accounting', 'Angular', 'C', 'C#', '...",0.4459,8.9181,97.9,Strong
4,2042775,https://remotive.com/remote-jobs/software-dev/...,Senior Software Engineer,EBSCO,https://remotive.com/job/2042775/logo,Software Development,"['C#', 'git', 'kubernetes', 'CI/CD', 'system a...",full_time,2025-08-02T16:51:11,USA,"usd 94,320 - 134,745 per year",EBSCO Information Services (EBSCO) delivers a ...,software-dev,"['.NET', 'API', 'AWS', 'Agile', 'C', 'C#', 'CI...",0.4342,8.6832,97.4,Strong
5,2044663,https://remotive.com/remote-jobs/devops/cloud-...,Cloud Engineer,Analytica,https://remotive.com/job/2044663/logo,DevOps / Sysadmin,"['AWS', 'python', 'CI/CD', 'terraform']",full_time,2025-08-13T10:50:35,USA,competitive compensation with bonuses and bene...,Analytica is seeking a Cloud Engineer with dee...,data-analysis,"['AWS', 'Amazon', 'Ansible', 'Bash', 'CI/CD', ...",0.4256,8.5122,97.0,Strong
6,2028991,https://remotive.com/remote-jobs/software-dev/...,Integrations Engineer / Java Developer,Omilia,https://remotive.com/job/2028991/logo,Software Development,"['git', 'java', 'sql', 'REST']",full_time,2025-07-03T18:50:07,Greece,NaN,An Integrations Engineer contributes to the de...,software-dev,"['Apple', 'Computer Science', 'Docker', 'Git',...",0.4249,8.4990,96.9,Strong
7,2048539,https://remotive.com/remote-jobs/software-dev/...,Java Fullstack Developer,Kastech Software Solutions Group.,https://remotive.com/job/2048539/logo,Software Development,"['elasticsearch', 'java']",contract,2025-08-16T16:50:54,Brazil,NaN,Skills/Qualifications: · Strong proficiency in...,software-dev,"['CI/CD', 'CSS', 'ES6', 'ElasticSearch', 'HTML...",0.4049,8.0988,95.7,Strong
8,1383531,https://remotive.com/remote-jobs/software-dev/...,Senior React Native Developer,Proxify,https://remotive.com/job/1383531/logo,Software Development,"['UI/UX', 'CI/CD']",full_time,2025-08-05T15:30:42,CET +/- 3 HOURS,$50k-$80K,The Role: We are looking for a Senior React Na...,software-dev,"['API', 'Apple', 'C', 'Java', 'React', 'React ...",0.3987,7.9747,95.2,Strong
9,2044815,https://remotive.com/remote-jobs/devops/aws-cl...,AWS Cloud DevOps Engineer,Redhorsecorp,https://remotive.com/job/2044815/logo,DevOps / Sysadmin,"['AWS', 'docker', 'kubernetes', 'CI/CD']",full_time,2025-08-10T12:52:06,USA,NaN,About the Organization Now is a great time to ...,data-analysis,"['AWS', 'A

In [123]:
df_out = df_jobs.copy()
if "tags" in df_out.columns:
    df_out["tags"] = df_out["tags"].apply(lambda x: "|".join(x) if isinstance(x, (list, tuple)) else x)
df_out.to_csv("jobs.csv", index=False, encoding="utf-8-sig")


In [124]:
df_jobs

,id,url,title,company_name,company_logo,category,tags,job_type,publication_date,candidate_required_location,salary,description,fetched_category,extracted_skills
0,2048457,https://remotive.com/remote-jobs/software-dev/...,Data Engineer,Kyivstar,https://remotive.com/job/2048457/logo,Software Development,"['apache', 'api', 'AWS', 'azure', 'big data', ...",full_time,2025-08-17T06:50:57,Ukraine,NaN,We are looking for a Data Engineer (NLP-Focuse...,software-dev,"['AWS', 'Analytics', 'Apache', 'Azure', 'Bash'..."
1,2048648,https://remotive.com/remote-jobs/software-dev/...,Senior Digital Engineer - Solution Architect,Capgemini,https://remotive.com/job/2048648/logo,Software Development,['automation'],full_time,2025-08-16T22:50:43,USA,NaN,About the job you’re considering We are lookin...,software-dev,"['C++', 'Healthcare', 'Infrastructure', 'Java'..."
2,2048458,https://remotive.com/remote-jobs/software-dev/...,Quantitative Developer,BitMEX,https://remotive.com/job/2048458/logo,Software Development,['java'],contract,2025-08-16T18:50:38,Asia,NaN,About BitMEX BitMEX stands as a globally leadi...,software-dev,"['Computer Science', 'Crypto', 'Docker', 'Java..."
3,2048460,https://remotive.com/remote-jobs/software-dev/...,Sr. Software Engineer - Front-End,Lively,https://remotive.com/job/2048460/logo,Software Development,"['javascript', 'node.js', 'postgresql', 'react']",full_time,2025-08-16T16:50:55,USA,"$125,000 - $200,000 usd",About Lively Lively set out to raise the bar o...,software-dev,"['API', 'AWS', 'Amazon', 'CSS', 'Computer Scie..."
4,2048539,https://remotive.com/remote-jobs/software-dev/...,Java Fullstack Developer,Kastech Software Solutions Group.,https://remotive.com/job/2048539/logo,Software Development,"['elasticsearch', 'java']",contract,2025-08-16T16:50:54,Brazil,NaN,Skills/Qualifications: · Strong proficiency in...,software-dev,"['CI/CD', 'CSS', 'ES6', 'ElasticSearch', 'HTML..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,2011400,https://remotive.com/remote-jobs/writing/paid-...,Paid Social Copywriter,Aristo Sourcing,https://remotive.com/job/2011400/logo,Writing,"['youtube', 'LAN']",full_time,2025-05-22T12:51:00,South Africa,"$2,000",Write compelling ad scripts and creative brief...,data-analysis,"['LAN', 'YouTube']"
1433,2011150,https://remotive.com/remote-jobs/design/smm-de...,SMM Design Team Lead,Neo Group,https://remotive.com/job/2011150/logo,Design,"['photoshop', 'Figma', 'graphic design']",full_time,2025-05-22T08:50:25,"Georgia, Bulgaria, Armenia, Kazakhstan",NaN,"Lead the SMM design team: generate ideas, prov...",data-analysis,"['Figma', 'Illustrator', 'Photoshop', 'motion ..."
1434,2011516,https://remotive.com/remote-jobs/writing/freel...,Freelance Technical Writer,Ng Voice,https://remotive.com/job/2011516/logo,Writing,"['troubleshooting', 'technical documentation']",freelance,2025-05-22T04:50:35,Worldwide,NaN,About us Committed to building the next genera...,data-analysis,"['API', 'documentation', 'infrastructure', 'on..."
1435,2011562,https://remotive.com/remote-jobs/sales-busines...,Account Executive,Diobox,https://remotive.com/job/2011562/logo,Sales / Business,['saas'],contract,2025-05-21T18:50:12,USA,NaN,"Diobox is a comprehensive, all-in-one platform...",data-analysis,"['Business Development', 'CRM', 'SaaS', 'start..."


In [137]:
from pathlib import Path

# 1) .gitignore
Path(".gitignore").write_text("""# Python
__pycache__/
*.py[cod]
.venv/
env/
.ipynb_checkpoints/
.mypy_cache/
.pytest_cache/
*.egg-info/

# OS
.DS_Store

# Data & artifacts
data/
artifacts/

# Secrets
.env
""", encoding="utf-8")

# 2) requirements.txt
reqs = [
    "python-dotenv",
    "openai",
    "pandas",
    "numpy",
    "scikit-learn",
    "sentence-transformers",
    "torch",
    "torch-geometric",
    "PyPDF2",
    "pdfminer.six",
    "docx2txt",
    "python-dateutil",
]
Path("requirements.txt").write_text("\n".join(reqs) + "\n", encoding="utf-8")

# 3) README.md (no triple backticks to avoid fence mistakes)
readme = (
    "# LinkPredictionJobs\n\n"
    "Jupyter notebook for link prediction / job recommendation experiments.\n\n"
    "## Setup\n"
    "Create a virtual environment, activate it, then install requirements:\n\n"
    "python -m venv .venv\n"
    "On Windows: .venv\\Scripts\\activate\n"
    "On macOS/Linux: source .venv/bin/activate\n"
    "pip install -r requirements.txt\n\n"
    "Keep secrets in a local `.env` (not committed) and load them with `python-dotenv`.\n"
)
Path("README.md").write_text(readme, encoding="utf-8")

print("Wrote: .gitignore, requirements.txt, README.md")


Wrote: .gitignore, requirements.txt, README.md


In [138]:
from pathlib import Path
print(Path('.').resolve())


C:\Users\Filip\Jupyter Notebook


In [141]:
!git grep -n "sk-" || echo none
!git grep -n "OPENAI_API_KEY" || echo none

Entity Recognition in Resumes.json:24:{"content": "Ijas Nizamuddin\nAssociate Consultant - State Street\n\nIrinchayam B.O, Kerala - Email me on Indeed: indeed.com/r/Ijas-\nNizamuddin/6748d77f76f94eed\n\nWith close to 3 years of experience in IT industry, I have had excellent exposure to design,\ndevelopment and implementation of Client Server Applications in various domains such as\nBanking and Finance concepts. I have been involved in various software Development projects\nin Open System environment.\n\nWORK EXPERIENCE\n\nAssociate Consultant\n\nOracle Corporation -\n\nJune 2011 to Present\n\nState Street Global Advisors (SSgA) is the asset management business of State Street\nCorporation, one of the world's leading providers of financial services to institutional investors1,\nwith a heritage dating back over two centuries. Backed by the strength and stability of the State\nStreet organization, SSgA makes continual investments in asset management and client service\nplatform, resultin

LinkPredictionJobs.ipynb:2653:      "jobs.csv:1420:2021218,https://remotive.com/remote-jobs/sales-business/revenue-operations-lead-2021218,Revenue Operations Lead,Zello,https://remotive.com/job/2021218/logo,Sales / Business,\"['sql', 'hubspot']\",full_time,2025-06-16T12:50:53,USA,,\"IMPORTANT: Please be aware, scammers may try to impersonate Zello by reaching out regarding job opportunities. We will never ask you for bank account information, checks, or other sensitive information as part of our hiring process. All correspondence will come from the zello.com email domain. If youÃ¢â‚¬â„¢re unsure, please email recruiting@zello.com with questions. About Zello Zello is a voice-first communication platform, powered by our industry-leading push-to-talk technology, to improve collaboration and productivity for desk-less workers. With over 175+ million users, weÃ¢â‚¬â„¢re the #1 rated push-to-talk app in the world, delivering 9 billion (yes, with a B) messages a month. At Zello, our company v

In [142]:
!git init
!git add LinkPredictionJobs.ipynb .gitignore requirements.txt README.md
!git commit -m "Initial commit: notebook + .gitignore + requirements + README"


Reinitialized existing Git repository in C:/Users/Filip/Jupyter Notebook/.git/


[main db569b1] Initial commit: notebook + .gitignore + requirements + README
 1 file changed, 94 insertions(+), 109 deletions(-)


In [143]:
!git status

On branch main
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   jobs.csv

no changes added to commit (use "git add" and/or "git commit -a")


In [144]:
!git branch -M main
!git remote add origin https://github.com/filipgjorgiev/job-recommendation.git

error: remote origin already exists.


In [146]:
!git remote set-url origin https://github.com/filipgjorgiev/job-recommendation.git

In [148]:
!git remote -v
!git remote get-url origin


origin	https://github.com/filipgjorgiev/job-recommendation.git (fetch)
origin	https://github.com/filipgjorgiev/job-recommendation.git (push)
https://github.com/filipgjorgiev/job-recommendation.git


In [149]:
!git rm -f --cached api-key.env
!git rm -f --cached "LinkPredictionJobs-Copy1.ipynb"


fatal: pathspec 'api-key.env' did not match any files
fatal: pathspec 'LinkPredictionJobs-Copy1.ipynb' did not match any files


In [151]:
!git ls-files



.gitignore
3. Data_Preparation.ipynb
Entity Recognition in Resumes.json
Exercise_3_full_code.ipynb
Exercise_5.ipynb
Exercise_6.ipynb
Exercise_7.ipynb
Filip_Gjorgiev CV.pdf
JobReccomendation_Remotive.ipynb
Laboratory exercise 2 - Introduction to Data Science (15.11.2024 12_30).ipynb
Laboratory_exercise_2.ipynb
LinkPredictionJobs.ipynb
README.md
blood_pressure_data.csv
cjgnn.pth
gpt_dataset.csv
jobs.csv
recommendations.csv
requirements.txt
scraped_data_jobs.csv
skill2id_train.pkl
soft2id_train.pkl
tenure_train_max.json
untitled.txt


In [153]:
!git rm -r --cached .

rm '.gitignore'
rm '3. Data_Preparation.ipynb'
rm 'Entity Recognition in Resumes.json'
rm 'Exercise_3_full_code.ipynb'
rm 'Exercise_5.ipynb'
rm 'Exercise_6.ipynb'
rm 'Exercise_7.ipynb'
rm 'Filip_Gjorgiev CV.pdf'
rm 'JobReccomendation_Remotive.ipynb'
rm 'Laboratory exercise 2 - Introduction to Data Science (15.11.2024 12_30).ipynb'
rm 'Laboratory_exercise_2.ipynb'
rm 'LinkPredictionJobs.ipynb'
rm 'README.md'
rm 'blood_pressure_data.csv'
rm 'cjgnn.pth'
rm 'gpt_dataset.csv'
rm 'jobs.csv'
rm 'recommendations.csv'
rm 'requirements.txt'
rm 'scraped_data_jobs.csv'
rm 'skill2id_train.pkl'
rm 'soft2id_train.pkl'
rm 'tenure_train_max.json'
rm 'untitled.txt'


In [154]:
!git status

On branch main
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	deleted:    .gitignore
	deleted:    3. Data_Preparation.ipynb
	deleted:    Entity Recognition in Resumes.json
	deleted:    Exercise_3_full_code.ipynb
	deleted:    Exercise_5.ipynb
	deleted:    Exercise_6.ipynb
	deleted:    Exercise_7.ipynb
	deleted:    Filip_Gjorgiev CV.pdf
	deleted:    JobReccomendation_Remotive.ipynb
	deleted:    Laboratory exercise 2 - Introduction to Data Science (15.11.2024 12_30).ipynb
	deleted:    Laboratory_exercise_2.ipynb
	deleted:    LinkPredictionJobs.ipynb
	deleted:    README.md
	deleted:    blood_pressure_data.csv
	deleted:    cjgnn.pth
	deleted:    gpt_dataset.csv
	deleted:    jobs.csv
	deleted:    recommendations.csv
	deleted:    requirements.txt
	deleted:    scraped_data_jobs.csv
	deleted:    skill2id_train.pkl
	deleted:    soft2id_train.pkl
	deleted:    tenure_train_max.json
	deleted:    untitled.txt

Untracked files:
  (use "git add <file>..." to include in w

In [155]:
!git add LinkPredictionJobs.ipynb .gitignore requirements.txt README.md

In [156]:
!git commit -m "Keep only notebook, .gitignore, requirements, and README"


[main 158de55] Keep only notebook, .gitignore, requirements, and README
 21 files changed, 181 insertions(+), 39060 deletions(-)
 delete mode 100644 3. Data_Preparation.ipynb
 delete mode 100644 Entity Recognition in Resumes.json
 delete mode 100644 Exercise_3_full_code.ipynb
 delete mode 100644 Exercise_5.ipynb
 delete mode 100644 Exercise_6.ipynb
 delete mode 100644 Exercise_7.ipynb
 delete mode 100644 Filip_Gjorgiev CV.pdf
 delete mode 100644 JobReccomendation_Remotive.ipynb
 delete mode 100644 Laboratory exercise 2 - Introduction to Data Science (15.11.2024 12_30).ipynb
 delete mode 100644 Laboratory_exercise_2.ipynb
 delete mode 100644 blood_pressure_data.csv
 delete mode 100644 cjgnn.pth
 delete mode 100644 gpt_dataset.csv
 delete mode 100644 jobs.csv
 delete mode 100644 recommendations.csv
 delete mode 100644 scraped_data_jobs.csv
 delete mode 100644 skill2id_train.pkl
 delete mode 100644 soft2id_train.pkl
 delete mode 100644 tenure_train_max.json
 delete mode 100644 untitled.tx

In [157]:
!git ls-files

.gitignore
LinkPredictionJobs.ipynb
README.md
requirements.txt


In [158]:
!git branch --show-current

main


In [159]:
!git push -u origin main

remote: error: GH013: Repository rule violations found for refs/heads/main.        
remote: 
remote: - GITHUB PUSH PROTECTION        
remote:   â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:     Resolve the following violations before pushing again        
remote: 
remote:     - Push cannot contain secrets        
remote: 
remote:             
remote:      (?) Learn how to resolve a blocked push        
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push        
remote:             
remote:             
remote:       â€”â€” OpenAI API Key â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”â€”        
remote:        locations:        
remote:          - commit: d3348731c2ef41aa02bcbf46832f7a331e87f184    

In [160]:
!git log

commit 158de553db34c8f4176e48a3b1ff71cd5ba5850a
Author: filipgjorgiev <filip.gjorgiev@outlook.finki.ukim.mk>
Date:   Thu Oct 2 10:21:12 2025 +0200

    Keep only notebook, .gitignore, requirements, and README

commit db569b192b320982c7790ed54ff4befce8f09179
Author: filipgjorgiev <filip.gjorgiev@outlook.finki.ukim.mk>
Date:   Thu Oct 2 10:06:34 2025 +0200

    Initial commit: notebook + .gitignore + requirements + README

commit d1d32c2d339bd2e8cccf682a1a7d47701d7b3560
Author: filipgjorgiev <filip.gjorgiev@outlook.finki.ukim.mk>
Date:   Mon Sep 29 21:54:01 2025 +0200

    Update notebook

commit fdde0823679231738544c5c741dd3b5678c807f6
Author: filipgjorgiev <filip.gjorgiev@outlook.finki.ukim.mk>
Date:   Mon Sep 29 21:27:57 2025 +0200

    Update notebook

commit e877779ec5056ab239cab2c445f515abc626215f
Author: filipgjorgiev <filip.gjorgiev@outlook.finki.ukim.mk>
Date:   Mon Sep 29 21:27:52 2025 +0200

    Initial commit: notebook + .gitignore + requirements + README

commit 1c86bb191ddc

In [162]:
!git branch -m main old-main

In [163]:
!git checkout --orphan main

M	LinkPredictionJobs.ipynb


Switched to a new branch 'main'


In [166]:
!git rm -r --cached -f .

rm '.gitignore'
rm 'LinkPredictionJobs.ipynb'
rm 'README.md'
rm 'requirements.txt'


In [167]:
!git add LinkPredictionJobs.ipynb .gitignore requirements.txt README.md

In [168]:
!git log

fatal: your current branch 'main' does not have any commits yet


In [171]:

!git status        


!git commit -m "Initial clean commit: notebook + essentials only"


!git log --oneline --decorate -n 3




On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)
[main (root-commit) 9a65c13] Initial clean commit: notebook + essentials only
 4 files changed, 3344 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 LinkPredictionJobs.ipynb
 create mode 100644 README.md
 create mode 100644 requirements.txt
9a65c13 (HEAD -> main) Initial clean commit: notebook + essentials only


In [172]:
!git push -u origin main --force

branch 'main' set up to track 'origin/main'.


To https://github.com/filipgjorgiev/job-recommendation.git
 * [new branch]      main -> main


In [174]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   LinkPredictionJobs.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	3. Data_Preparation.ipynb
	Entity Recognition in Resumes.json
	Exercise_3_full_code.ipynb
	Exercise_5.ipynb
	Exercise_6.ipynb
	Exercise_7.ipynb
	Filip_Gjorgiev CV.pdf
	JobReccomendation_Remotive.ipynb
	Laboratory exercise 2 - Introduction to Data Science (15.11.2024 12_30).ipynb
	Laboratory_exercise_2.ipynb
	blood_pressure_data.csv
	cjgnn.pth
	gpt_dataset.csv
	jobs.csv
	recommendations.csv
	scraped_data_jobs.csv
	skill2id_train.pkl
	soft2id_train.pkl
	tenure_train_max.json
	untitled.txt

no changes added to commit (use "git add" and/or "git commit -a")
